In [190]:
# import the library
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import torch.nn.functional as F
from sklearn.preprocessing import LabelBinarizer
import sys
import csv
import pandas as pd
from torchvision.io import read_image
from sklearn.metrics import accuracy_score
from PIL import Image

In [191]:
np.set_printoptions(threshold=sys.maxsize)

In [192]:
classes = []
csv_file = './pokemon.csv'
with open(csv_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row if present
    for row in reader:
        classes.append(row[1])
       # classes.append(row[2])
        
unique_types = np.unique(classes)
print(unique_types)
encoder = LabelBinarizer()
testOneHot = encoder.fit_transform(classes)
dataroot = os.getcwd() + '/images/'
print(testOneHot)



# oneHot = f.one_hot(torch.tensor(classes), num_classes = 18)
# print(np.unique(classes))



['Bug' 'Dark' 'Dragon' 'Electric' 'Fairy' 'Fighting' 'Fire' 'Flying'
 'Ghost' 'Grass' 'Ground' 'Ice' 'Normal' 'Poison' 'Psychic' 'Rock' 'Steel'
 'Water']
[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1

In [193]:
# Reference: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
class MyDataSet():
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_# mage(img_path + ".png")
        label = self.img_labels.iloc[idx, 1]
        image = Image.open(img_path + ".png").convert('RGB')
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label#

In [194]:
image_transform = transforms.Compose([transforms.Resize((128, 128)), 
                                        transforms.ToPILImage(),
                                        transforms.ToTensor()
                                        ])

pokemon_dataset = MyDataSet('./pokemon.csv', './images/pokemon2/', transform = image_transform)

pokemon_dataloader = torch.utils.data.DataLoader(pokemon_dataset, batch_size = 32, shuffle = True)

In [195]:

# Number of training epochs
num_epochs = 500

# Learning rate for optimizers
lr = 1e-3

In [196]:
#model setup default
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


classifier = Classifier()

In [197]:
optimizer = torch.optim.Adam(classifier.parameters(), lr = lr)
loss_function = nn.CrossEntropyLoss()

In [198]:
#train loop
loss = []
for epoch in range(num_epochs):
    for i, dat in enumerate(pokemon_dataloader, 0):
       # print(dat)
        X, y = dat #x_trian, y_true
        print(X)

        #zero the gradient
        optimizer.zero_grad()

        #get pred
        y_pred = classifier(X)

        loss.append(loss_function(y_pred, y))
        #back prop and move next image
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print("epoch: ", i)
            print("loss", loss[i])
            print("accuracy: ", accuracy_score(y, y_pred))
            


TypeError: pic should be Tensor or ndarray. Got <class 'PIL.Image.Image'>.